In [ ]:
 # from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
# cd drive/MyDrive/따릉이

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

import warnings
warnings.filterwarnings('ignore')


plt.rc('font', family='Malgun Gothic')

In [ ]:
station_df = pd.read_csv('data/station_data.csv', index_col=0)

In [ ]:
break_df = pd.read_csv('data/서울시 공공자전거 고장신고 내역_23.1-6.csv', encoding="cp949")

In [ ]:
office_df = pd.read_csv('data/서울특별시 공공자전거 대여소별 이용정보(월별)_23.1-6.csv', encoding="cp949")

In [ ]:
history_df = pd.read_csv('data/서울특별시 공공자전거 대여이력 정보_2301.csv', encoding="cp949")

In [ ]:
station_df.info()

In [ ]:
break_df.info()

In [ ]:
history_df.info()

In [ ]:
history_df['대여 대여소번호'][0] == history_df['반납대여소번호'][0]

In [ ]:
history_df['대여 대여소번호'][0]

In [ ]:
history_df['반납대여소번호'] = history_df['반납대여소번호'].replace("\\N", np.nan)
history_df['반납대여소번호'] = history_df['반납대여소번호'].astype(float)
history_df['반납대여소번호']

In [ ]:
office_df.info()

In [ ]:
office_df['대여건수'] = office_df['대여건수'].str.replace(",", "").astype(int)

In [ ]:
office_df['반납건수'] = office_df['반납건수'].str.replace(",", "").replace(' - ', 0).astype(int)

In [ ]:
station_df.columns

In [ ]:
office_df.columns

In [ ]:
merged_df = office_df.merge(station_df, left_on="대여소명", right_on='stationName', how="inner")
merged_df.head()

In [ ]:
january_data = merged_df[merged_df['기준년월'] == 202301]

In [ ]:
mean_lat = january_data.drop_duplicates('대여소명')['stationLatitude'].mean()
mean_lng = january_data.drop_duplicates('대여소명')['stationLongitude'].mean()

In [ ]:
# !pip install folium

In [ ]:
import folium

In [ ]:
def compute_color(value, min_val, max_val):
    value = np.log1p(value)
    min_val = np.log1p(min_val)
    max_val = np.log1p(max_val)

    normed_value = (value - min_val) / (max_val - min_val)
    cmap = plt.get_cmap("YlOrRd")
    (r,g,b,a) = cmap(normed_value)
    return f"#{int(r*255):02X}{int(g*255):02X}{int(b*255):02X}"

In [ ]:
min_usage = january_data['대여건수'].min()
max_usage = january_data['대여건수'].max()

m = folium.Map(location=[mean_lat, mean_lng], zoom_start=14)

for idx, row in january_data.iterrows():
    location = [row['stationLatitude'], row['stationLongitude']]
    usage = row['대여건수']
    color = compute_color(usage, min_usage, max_usage)

    tooltip_text = f"대여소명: {row['대여소명']}<br>대여건수: {row['대여건수']}<br>반납건수: {row['반납건수']}"
    tooltip = folium.Tooltip(tooltip_text)

    folium.CircleMarker(
        location=location,
        radius=5,
        color=color,
        tooltip=tooltip
    ).add_to(m)

m

In [ ]:
min_usage_rental = january_data['대여건수'].min()
max_usage_rental = january_data['대여건수'].max()

min_usage_return = january_data['반납건수'].min()
max_usage_return = january_data['반납건수'].max()

m = folium.Map(location=[mean_lat, mean_lng], zoom_start=14)

for idx, row in january_data.iterrows():
    location = [row['stationLatitude'], row['stationLongitude']]

    usage_rental = row['대여건수']
    color = compute_color(usage_rental, min_usage_rental, max_usage_rental)

    usage_return = row['반납건수']
    fill_color = compute_color(usage_return, min_usage_return, max_usage_return)

    tooltip_text = f"대여소명: {row['대여소명']}<br>대여건수: {row['대여건수']}<br>반납건수: {row['반납건수']}"
    tooltip = folium.Tooltip(tooltip_text)

    folium.CircleMarker(
        location=location,
        radius=5,
        color=color,
        tooltip=tooltip,
        fill=True,
        fill_color=fill_color,
        fill_opacity=1
    ).add_to(m)

m

In [ ]:
from ipywidgets import interact

In [ ]:
months = [202301, 202302, 202303, 202304, 202305, 202306]
maps = {}
min_usage_rental = merged_df['대여건수'].min()
max_usage_rental = merged_df['대여건수'].max()

min_usage_return = merged_df['반납건수'].min()
max_usage_return = merged_df['반납건수'].max()

mean_lat = merged_df.drop_duplicates('대여소명')['stationLatitude'].mean()
mean_lng = merged_df.drop_duplicates('대여소명')['stationLongitude'].mean()

for month in months:
    month_data = merged_df[merged_df['기준년월'] == month]
    m = folium.Map(location=[mean_lat, mean_lng], zoom_start=14)

    for idx, row in month_data.iterrows():
        location = [row['stationLatitude'], row['stationLongitude']]

        usage_rental = row['대여건수']
        color = compute_color(usage_rental, min_usage_rental, max_usage_rental)

        usage_return = row['반납건수']
        fill_color = compute_color(usage_return, min_usage_return, max_usage_return)

        tooltip_text = f"대여소명: {row['대여소명']}<br>대여건수: {row['대여건수']}<br>반납건수: {row['반납건수']}"
        tooltip = folium.Tooltip(tooltip_text)

        folium.CircleMarker(
            location=location,
            radius=5,
            color=color,
            tooltip=tooltip,
            fill=True,
            fill_color=fill_color,
            fill_opacity=1
        ).add_to(m)

    maps[month] = m

In [ ]:
def show_map(month):
    return maps[month]

In [ ]:
interact(show_map, month=months) # list를 활용한 interact 활용

In [ ]:
interact(show_map, month=(202301, 202306, 1)) # tuple을 활용한 interact 활용

In [ ]:
# !pip install plotly.express

In [ ]:
import plotly.express as px
merged_df['반납건수'] = merged_df['반납건수'].fillna(0)

fig = px.scatter(merged_df,
                hover_name="대여소명",
                 x='stationLongitude',
                 y='stationLatitude',
                 color='대여건수',
                 size='반납건수',
                 animation_frame='기준년월',
                 range_color=[merged_df['대여건수'].min(), merged_df['대여건수'].max()],
                 )

fig.update_layout(width=1000, height=1000)
fig.show()

## history_df활용

### 시간대별 이용건수

In [ ]:
history_df['대여시간'] = pd.to_datetime(history_df['대여일시']).dt.hour
history_df.head()

In [ ]:
hourly_history_counts = history_df['대여시간'].value_counts().sort_index()
hourly_history_counts

In [ ]:
plt.figure(figsize=(12, 6))
hourly_history_counts.plot(kind='bar', color='skyblue')
plt.xticks(rotation=0)
plt.grid(axis='y')
plt.show()

### 시간대 별 이용시간

In [ ]:
duration_mean_by_hour = history_df.groupby('대여시간')['이용시간(분)'].mean()
duration_mean_by_hour

In [ ]:
plt.figure(figsize=(12, 6))
duration_mean_by_hour.plot(kind='bar', color='pink')
plt.xticks(rotation=0)
plt.grid(axis='y')
plt.show()

### 시간대 별 이용거리

In [ ]:
history_df.head()

In [ ]:
distance_mean_by_hour = history_df.groupby('대여시간')['이용거리(M)'].mean()
distance_mean_by_hour

In [ ]:
plt.figure(figsize=(12, 6))
distance_mean_by_hour.plot(kind='bar', color='lightgreen')
plt.xticks(rotation=0)
plt.grid(axis='y')
plt.show()

### 성별별 이용량

In [ ]:
history_df['성별'].unique()

In [ ]:
history_df['성별'] = history_df['성별'].replace({'m':'M', 'f':'F', np.nan:'\\N'})

In [ ]:
gender_counts = history_df.groupby('성별').size()
gender_counts

In [ ]:
gender_counts.plot(kind='pie', autopct='%1.1f%%', startangle=90)
plt.show()

In [ ]:
gender_counts_by_hour = history_df.groupby(['대여시간', '성별']).size().unstack().fillna(0)
gender_counts_by_hour

In [ ]:
colors = ['pink', 'skyblue','gray']
gender_counts_by_hour.plot(kind='bar', stacked=True, color=colors, width=0.7)
plt.xticks(rotation=0)
plt.grid(axis='y')
plt.show()

In [ ]:
gender_ratios_by_hour = (gender_counts_by_hour.T/gender_counts_by_hour.sum(axis=1)).T
gender_ratios_by_hour

In [ ]:
colors = ['pink', 'skyblue','gray']
gender_ratios_by_hour.plot(kind='bar', stacked=True, color=colors, width=0.7)
plt.xticks(rotation=0)
plt.grid(axis='y')
plt.show()

### 연령대별 이용량

In [ ]:
history_df['생년'].unique()

In [ ]:
history_df['생년'] = history_df['생년'].replace('\\N', np.nan)
history_df['생년'] = history_df['생년'].astype(float)
history_df['생년']

In [ ]:
import datetime

In [ ]:
current_year = datetime.datetime.now().year
current_year

In [ ]:
filtered_history_df = history_df.copy()
filtered_history_df = filtered_history_df[(filtered_history_df['생년'] > 0) & (filtered_history_df['생년'] <= 2010)]

In [ ]:
filtered_history_df['연령'] = current_year - filtered_history_df['생년']
filtered_history_df

In [ ]:
age_bins = [13, 20, 30, 40, 50, 60, 70, 80, 90, 100]
age_labels = ['13-19', '20-29', '30-39', '40-49', '50-59', '60-69', '70-79', '80-89', '90+']

filtered_history_df['연령대'] = pd.cut(filtered_history_df['연령'], bins=age_bins, labels=age_labels, right=False)
filtered_history_df

In [ ]:
age_counts = filtered_history_df['연령대'].value_counts().sort_index()
age_counts

In [ ]:
plt.figure(figsize=(8, 6))
age_counts.plot(kind='pie', autopct='%1.1f%%', startangle=90)
plt.show()

In [ ]:
age_counts_by_hour = filtered_history_df.groupby(['대여시간', '연령대']).size().unstack().fillna(0)

plt.figure(figsize = (16,8))
age_counts_by_hour.plot(kind='bar', stacked=True, figsize=(14,7), cmap="rainbow_r", width=0.7)
plt.xticks(rotation=0)
plt.grid(axis='y')
plt.show()

### 주말, 평일 비교

In [ ]:
history_df['대여일'] = pd.to_datetime(history_df['대여일시']).dt.date
history_df['대여일'] = pd.to_datetime(history_df['대여일'])
history_df.head()

In [ ]:
history_df['주말'] = history_df['대여일'].dt.weekday >= 5
history_df

In [ ]:
duration_mean_weekday = history_df[~history_df['주말']]['이용시간(분)'].mean()
duration_mean_weekend = history_df[history_df['주말']]['이용시간(분)'].mean()

distance_mean_weekday = history_df[~history_df['주말']]['이용거리(M)'].mean()
distance_mean_weekend = history_df[history_df['주말']]['이용거리(M)'].mean()

daily_counts_weekday = history_df[~history_df['주말']].groupby('대여일').size().mean()
daily_counts_weekend = history_df[history_df['주말']].groupby('대여일').size().mean()

In [ ]:
duration_mean_weekday, duration_mean_weekend, distance_mean_weekday, distance_mean_weekend, daily_counts_weekday, daily_counts_weekend

In [ ]:
grouped_data = history_df.groupby(['주말', '대여시간']).size().unstack(level=0)

plt.figure(figsize=(12, 6))
grouped_data.plot(kind='bar', color=['coral', 'skyblue'])
plt.xticks(rotation=0)
plt.legend(['Weekday', 'Weekend'])
plt.show()

In [ ]:
grouped_data = history_df.groupby(['주말', '대여시간']).mean()[['이용시간(분)']].unstack(level=0)

plt.figure(figsize=(12, 6))
grouped_data.plot(kind='bar', color=['coral', 'skyblue'])
plt.xticks(rotation=0)
plt.legend(['Weekday', 'Weekend'])
plt.show()

### 주말, 평일 연령대 비교

In [ ]:
filtered_history_df['대여일'] = pd.to_datetime(filtered_history_df['대여일시']).dt.date
filtered_history_df['대여일'] = pd.to_datetime(filtered_history_df['대여일'])
filtered_history_df['대여일']

In [ ]:
filtered_history_df['주말'] = filtered_history_df['대여일'].dt.weekday >= 5  # 5 and 6 correspond to Saturday and Sunday
filtered_history_df['주말']

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(15, 6))

# 주중 데이터
weekday_data = filtered_history_df[filtered_history_df['주말'] == False]
weekday_age_counts = weekday_data['연령대'].value_counts().sort_index()
weekday_age_counts.plot(kind='pie', autopct='%1.1f%%', startangle=90, ax=ax[0], title="주중")

# 주말 데이터
weekend_data = filtered_history_df[filtered_history_df['주말'] == True]
weekend_age_counts = weekend_data['연령대'].value_counts().sort_index()
weekend_age_counts.plot(kind='pie', autopct='%1.1f%%', startangle=90, ax=ax[1], title="주말")

plt.tight_layout()
plt.show()

In [ ]:
grouped_data = filtered_history_df.groupby(['주말', '대여시간', '연령대']).size().unstack(level=0)

fig, ax = plt.subplots(figsize=(10,6))

# 주말 데이터 스택
grouped_data[True].unstack(level='연령대').plot(kind='bar', stacked=True, position=0, width=0.4, ax=ax)

# 주중 데이터 스택
grouped_data[False].unstack(level='연령대').plot(kind='bar', stacked=True, position=1, width=0.4, ax=ax, legend=False )

plt.show()


### history_df scatter에 정보 나타내기 (시간 흐름에 따른 대여건수 변화 시각화해보기)

In [ ]:
history_df['대여일'] =  history_df['대여일'].dt.date

In [ ]:
grouped_rentals_df = history_df.groupby(['대여일', '대여시간', '대여 대여소명']).size().reset_index(name='대여건수')
grouped_rentals_df.head()

In [ ]:
station_df['대여 대여소명'] = station_df['stationName'].str.split('.').str[1].str.strip()
station_df.head()

In [ ]:
merged_data = pd.merge(grouped_rentals_df, station_df, on='대여 대여소명', how='inner')
merged_data.head()

In [ ]:
merged_data.columns

In [ ]:
date_groupby_df = merged_data.groupby(['대여일', '대여 대여소명']).agg({
    '대여건수':'sum',
    "stationLatitude":'first',
    "stationLongitude":'first'
}).reset_index()
date_groupby_df.head()

In [ ]:
date_groupby_df.columns

In [ ]:
def get_range_color(s):
    q1 = s.quantile(0.25)
    q3 = s.quantile(0.75)
    IQR = q3 - q1
    lower_bound = q1 - 1.5 * IQR
    upper_bound = q3 + 1.5 * IQR

    range_color = [
        max(s.min(), lower_bound),
        min(s.max(), upper_bound)
    ]

    return range_color

In [ ]:
fig = px.scatter(date_groupby_df,
                x="stationLongitude",
                y="stationLatitude",
                animation_frame="대여일",
                range_color= get_range_color(date_groupby_df['대여건수']),
                hover_name="대여 대여소명",
                color="대여건수")
fig.show()

In [ ]:
date_groupby_df.head()

In [ ]:
pivot_df = date_groupby_df.pivot_table(index='대여일', columns='대여 대여소명', values='대여건수', fill_value=0).reset_index()
pivot_df.head()

In [ ]:
reshaped_df = pivot_df.melt(id_vars=['대여일'], value_name='대여건수', var_name='대여 대여소명')
reshaped_df.head()

In [ ]:
date_groupby_df = reshaped_df.merge((date_groupby_df[['대여 대여소명', "stationLatitude", "stationLongitude"]]).drop_duplicates())
date_groupby_df.head()

In [ ]:
fig = px.scatter(date_groupby_df,
                x="stationLongitude",
                y="stationLatitude",
                animation_frame="대여일",
                range_color= get_range_color(date_groupby_df['대여건수']),
                hover_name="대여 대여소명",
                color="대여건수")

fig.update_layout(autosize=False, width=1000, height=600)
fig.show()

In [ ]:
hour_groupby_df = merged_data.groupby(['대여시간', '대여 대여소명']).agg({
    '대여건수':'sum',
    'stationLatitude':'first',
    'stationLongitude':'first'
}).reset_index()
hour_groupby_df.head()

In [ ]:
pivot_df = hour_groupby_df.pivot_table(index='대여시간', columns='대여 대여소명', values='대여건수', fill_value=0).reset_index()
reshaped_df = pivot_df.melt(id_vars=['대여시간'], value_name='대여건수', var_name='대여 대여소명')
hour_groupby_df = reshaped_df.merge(hour_groupby_df[['대여 대여소명', "stationLatitude", "stationLongitude"]].drop_duplicates())
hour_groupby_df.head()

In [ ]:
fig = px.scatter(hour_groupby_df,
                x="stationLongitude",
                y="stationLatitude",
                animation_frame="대여시간",
                range_color= get_range_color(hour_groupby_df['대여건수']),
                hover_name="대여 대여소명",
                color="대여건수",
                color_continuous_scale=px.colors.sequential.Reds
                )

fig.update_layout(autosize=False, width=1000, height=600)
fig.show()